In [1]:
import json
import os
from typing import Optional, Tuple, List
from datetime import datetime
from pathlib import Path
from openai import OpenAI
import fire
import pandas as pd

import numpy as np
from sb3_contrib.ppo_mask import MaskablePPO
from stable_baselines3.common.callbacks import BaseCallback

from alphagen.data.expression import *
from alphagen.data.parser import ExpressionParser
from alphagen.models.linear_alpha_pool import LinearAlphaPool, MseAlphaPool
from alphagen.rl.env.wrapper import AlphaEnv
from alphagen.rl.policy import LSTMSharedNet
from alphagen.utils import reseed_everything, get_logger
from alphagen.rl.env.core import AlphaEnvCore
from alphagen_qlib.calculator import QLibStockDataCalculator
from alphagen_qlib.stock_data import initialize_qlib
from alphagen_llm.client import ChatClient, OpenAIClient, ChatConfig
from alphagen_llm.prompts.system_prompt import EXPLAIN_WITH_TEXT_DESC
from alphagen_llm.prompts.interaction import InterativeSession, DefaultInteraction

In [2]:
instruments: str = "csi300"
device = torch.device("cuda:0")


def get_dataset(start: str, end: str) -> StockData:
    return StockData(
        instrument=instruments,
        start_time=start,
        end_time=end,
        device=device
    )

segments = [
    ("2012-01-01", "2019-12-31"),
    ("2022-01-01", "2022-06-30"),
    ("2022-07-01", "2022-12-31"),
    ("2023-01-01", "2023-06-30")
]


datasets = [get_dataset(*s) for s in segments]

[23512:MainThread](2025-04-24 12:33:13,832) INFO - qlib.Initialization - [config.py:420] - default_conf: client.
[23512:MainThread](2025-04-24 12:33:14,950) INFO - qlib.Initialization - [__init__.py:74] - qlib successfully initialized based on client settings.
[23512:MainThread](2025-04-24 12:33:14,952) INFO - qlib.Initialization - [__init__.py:76] - data_path={'__DEFAULT_FREQ': WindowsPath('C:/Users/tywat/.qlib/qlib_data/cn_data')}


In [3]:
close = Feature(FeatureType.CLOSE)
target = Ref(close, -20) / close - 1
calculators = [QLibStockDataCalculator(d, target) for d in datasets]

In [4]:
from alphagen.data.expression import Operators
from alphagen.data.parser import ExpressionParser

def load_linear_alpha_pool_from_json(json_path: str, 
                                     calculator: QLibStockDataCalculator,
                                     single_alpha: bool = False) -> LinearAlphaPool | list[LinearAlphaPool]:
    # Load the JSON file
    parser = ExpressionParser(Operators)
    with open(json_path, 'r') as f:
        pool_data = json.load(f)

    # Extract expressions and weights from the loaded data
    expressions = pool_data['exprs']
    weights = pool_data['weights']

    # Create an instance of LinearAlphaPool
    alpha_pool = MseAlphaPool(
        capacity=len(expressions),  # Set the capacity based on the number of expressions
        calculator=calculator
    )

    # Load the expressions into the pool
    expres = []
    if single_alpha:
        alpha_pools = []

        for expression,weight in zip(expressions,weights):
            alpha_pool = MseAlphaPool(
                capacity=1,
                calculator=calculator
                )
            expre = parser.parse(expression)
            alpha_pool.force_load_exprs([expre], [weight])
            alpha_pools.append(alpha_pool)

        return  alpha_pools
    else:
        for expression in expressions:
            expre = parser.parse(expression)
            expres.append(expre)
        
        
        alpha_pool.force_load_exprs(expres, weights)

        return alpha_pool

alpha_pools = load_linear_alpha_pool_from_json('out/results/csi300_20_0_20250208124320_rl/251904_steps_pool.json', calculators[1])
alpha_pool = load_linear_alpha_pool_from_json('out/results/csi300_20_0_20250208124320_rl/251904_steps_pool.json', calculators[1], single_alpha=True)

In [5]:
ic_value, rank_ic_value = alpha_pools.test_ensemble(calculators[2])
print(alpha_pools.exprs)
print(ic_value, rank_ic_value)

[Greater(Div(Div(-1.0,$high),EMA($open,10d)),-2.0), Delta(Log($vwap),1d), Mul($volume,Mul(Cov($close,Mul(5.0,Min(Mul($high,-30.0),40d)),40d),-0.01)), Sum(Mul(Corr(Div($vwap,-0.5),$close,5d),-10.0),10d), Abs(Sub(2.0,Div($close,Add(Greater(2.0,Delta(Log($low),5d)),30.0)))), Mad(Add(2.0,Mean($vwap,20d)),10d), Corr($close,$low,10d), Abs(Log(Mad(Sub(-0.5,$close),20d))), Mad(Log(Log($volume)),40d), Mul(0.5,Corr(Log($volume),WMA(Log($volume),40d),40d)), Mul(Mul($volume,Mul(Add(Mean($high,20d),30.0),$high)),0.5), Mul(WMA(Log(Abs(Var($low,5d))),20d),-2.0), Abs(Mul(5.0,Sub($open,30.0))), Mean(Less(Sub(-2.0,Corr($volume,$high,20d)),1.0),10d), Sub(Less(1.0,$low),5.0), Add(Corr(Sub(-1.0,$high),$volume,10d),0.01), WMA(Div(Std(WMA(Div(Div($vwap,30.0),$low),40d),20d),-5.0),10d), WMA(Sub(-1.0,Div($low,$close)),20d), Less(Div($close,$vwap),$volume), Sub(Mad(Mean(Log($low),20d),40d),5.0), None]
0.06614601612091064 0.0644562840461731


In [6]:
alpha_index = 3

ic_value, rank_ic_value = alpha_pool[alpha_index].test_ensemble(calculators[2])
print(alpha_pool[alpha_index].exprs)
print(ic_value, rank_ic_value)

[Sum(Mul(Corr(Div($vwap,-0.5),$close,5d),-10.0),10d), None]
0.010267447680234909 0.010892813093960285


In [7]:
for alpha in alpha_pool:
    print(alpha.exprs)

[Greater(Div(Div(-1.0,$high),EMA($open,10d)),-2.0), None]
[Delta(Log($vwap),1d), None]
[Mul($volume,Mul(Cov($close,Mul(5.0,Min(Mul($high,-30.0),40d)),40d),-0.01)), None]
[Sum(Mul(Corr(Div($vwap,-0.5),$close,5d),-10.0),10d), None]
[Abs(Sub(2.0,Div($close,Add(Greater(2.0,Delta(Log($low),5d)),30.0)))), None]
[Mad(Add(2.0,Mean($vwap,20d)),10d), None]
[Corr($close,$low,10d), None]
[Abs(Log(Mad(Sub(-0.5,$close),20d))), None]
[Mad(Log(Log($volume)),40d), None]
[Mul(0.5,Corr(Log($volume),WMA(Log($volume),40d),40d)), None]
[Mul(Mul($volume,Mul(Add(Mean($high,20d),30.0),$high)),0.5), None]
[Mul(WMA(Log(Abs(Var($low,5d))),20d),-2.0), None]
[Abs(Mul(5.0,Sub($open,30.0))), None]
[Mean(Less(Sub(-2.0,Corr($volume,$high,20d)),1.0),10d), None]
[Sub(Less(1.0,$low),5.0), None]
[Add(Corr(Sub(-1.0,$high),$volume,10d),0.01), None]
[WMA(Div(Std(WMA(Div(Div($vwap,30.0),$low),40d),20d),-5.0),10d), None]
[WMA(Sub(-1.0,Div($low,$close)),20d), None]
[Less(Div($close,$vwap),$volume), None]
[Sub(Mad(Mean(Log($low),

In [8]:
ics = []
rank_ics = []
alphas = []

for alpha in alpha_pool:
    ic_value, rank_ic_value = alpha.test_ensemble(calculators[2])

    ics.append(ic_value)
    rank_ics.append(rank_ic_value)
    alphas.append(alpha.exprs)

df_ic_ind = pd.DataFrame({'alpha': alphas, 'ic': ics, 'rank_ic': rank_ics})
df_ic_ind

,alpha,ic,rank_ic
0,"[Greater(Div(Div(-1.0,$high),EMA($open,10d)),-...",0.055708,0.084874
1,"[Delta(Log($vwap),1d), None]",-0.024560,-0.012933
2,"[Mul($volume,Mul(Cov($close,Mul(5.0,Min(Mul($h...",-0.036502,-0.035545
3,"[Sum(Mul(Corr(Div($vwap,-0.5),$close,5d),-10.0...",0.010267,0.010893
4,"[Abs(Sub(2.0,Div($close,Add(Greater(2.0,Delta(...",-0.061593,-0.092103
5,"[Mad(Add(2.0,Mean($vwap,20d)),10d), None]",-0.006636,0.043511
6,"[Corr($close,$low,10d), None]",0.056122,0.063845
7,"[Abs(Log(Mad(Sub(-0.5,$close),20d))), None]",-0.081099,-0.097603
8,"[Mad(Log(Log($volume)),40d), None]",-0.025405,-0.042726
9,"[Mul(0.5,Corr(Log($volume),WMA(Log($volume),40...",-0.040783,-0.046877


In [9]:
for p in Path("out/gp").iterdir():
    seed = int(p.name)

with open(p / "40.json") as f:
    report = json.load(f)


state = report["res"]["res"]["pool_state"]
state["exprs"]

['Std(EMA(Min(Mul(5.0,$high),30d),50d),10d)',
 'Std(EMA(Min(Log($vwap),30d),40d),10d)',
 'Sum(Mean(Abs(Corr($low,$high,20d)),40d),20d)',
 'Std(Med(Min(Mul(5.0,$high),30d),10d),10d)',
 'Mad(Min($low,20d),20d)',
 'Std(Std(Min(Mul($vwap,2.0),30d),10d),10d)',
 'Mad(Med($close,50d),10d)',
 'Std(Cov(Corr(Var($volume,40d),$high,20d),$close,30d),10d)',
 'Std(Med(Ref(Mul(10.0,$high),30d),10d),10d)',
 'Std(Min(Sum(Mul(10.0,$high),40d),50d),10d)',
 'Mad(Min($high,30d),10d)',
 'Std(Std(Med(Mul(0.5,$high),20d),20d),10d)',
 'Mad(Ref(Min($high,30d),10d),10d)',
 'Std(Abs(WMA(Cov(0.01,$high,50d),10d)),10d)',
 'Std(Min(WMA($high,20d),50d),10d)',
 'Log(Var(Sum($low,30d),40d))',
 'Std(EMA(Min(Mul(Std($high,10d),$high),30d),50d),10d)',
 'Std(Max(Min(Mul(5.0,$high),30d),20d),10d)',
 'Std(Min(Mean(Corr(5.0,$high,30d),50d),10d),10d)',
 'Std(EMA(WMA($vwap,10d),40d),10d)']

# main

In [49]:
ex_num = "51-5"

In [50]:
import pickle

file_path = f'out/backtests/{ex_num}/gp/2-graph.pkl'

with open(file_path, 'rb') as file:
    chart = pickle.load(file)
chart.show()

In [51]:
import pickle

file_path = f'out/backtests/{ex_num}/gp/2-report.pkl'

with open(file_path, 'rb') as file:
    gp_report = pickle.load(file)
gp_report

,account,return,total_turnover,turnover,total_cost,cost,value,cash,bench
datetime,,,,,,,,,
2022-07-01,1.000000e+08,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,1.000000e+08,-0.004078
2022-07-04,9.985793e+07,-1.792796e-16,9.471627e+07,0.947163,1.420744e+05,0.001421,9.471627e+07,5.141652e+06,0.006562
2022-07-05,9.888758e+07,-9.364478e-03,1.182004e+08,0.235176,1.773006e+05,0.000353,9.813944e+07,7.481377e+05,-0.001444
2022-07-06,9.692611e+07,-1.960765e-02,1.332146e+08,0.151831,1.998219e+05,0.000228,9.652703e+07,3.990815e+05,-0.014605
2022-07-07,9.751226e+07,6.279970e-03,1.482389e+08,0.155008,2.223584e+05,0.000233,9.710154e+07,4.107244e+05,0.004408
...,...,...,...,...,...,...,...,...,...
2023-12-25,6.861795e+07,6.954879e-03,3.623637e+09,0.076389,5.435455e+06,0.000115,6.848075e+07,1.372027e+05,0.003062
2023-12-26,6.806847e+07,-7.887691e-03,3.629131e+09,0.080080,5.443697e+06,0.000120,6.792749e+07,1.409804e+05,-0.006769
2023-12-27,6.800730e+07,-7.787308e-04,3.634571e+09,0.079909,5.451856e+06,0.000120,6.786620e+07,1.411080e+05,0.003480


In [52]:
import pickle

file_path = f'out/backtests/{ex_num}/rl/0-graph.pkl'

with open(file_path, 'rb') as file:
    chart = pickle.load(file)
chart.show()

In [53]:
import pickle

file_path = f'out/backtests/{ex_num}/rl/0-report.pkl'

with open(file_path, 'rb') as file:
    alphaGen_report = pickle.load(file)
alphaGen_report

,account,return,total_turnover,turnover,total_cost,cost,value,cash,bench
datetime,,,,,,,,,
2022-07-01,1.000000e+08,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,1.000000e+08,-0.004078
2022-07-04,9.985780e+07,-4.778849e-16,9.480169e+07,0.948017,1.422025e+05,0.001422,9.480169e+07,5.056106e+06,0.006562
2022-07-05,9.914609e+07,-6.782694e-03,1.177376e+08,0.229686,1.766064e+05,0.000345,9.844103e+07,7.050580e+05,-0.001444
2022-07-06,9.773149e+07,-1.396534e-02,1.377296e+08,0.201641,2.065944e+05,0.000302,9.720178e+07,5.297165e+05,-0.014605
2022-07-07,9.828185e+07,5.930105e-03,1.571936e+08,0.199158,2.357904e+05,0.000299,9.774347e+07,5.383801e+05,0.004408
...,...,...,...,...,...,...,...,...,...
2023-12-25,6.592298e+07,9.081655e-03,6.234346e+09,0.198622,9.351519e+06,0.000298,6.558608e+07,3.368954e+05,0.003062
2023-12-26,6.545343e+07,-6.832115e-03,6.247113e+09,0.193660,9.370669e+06,0.000290,6.512052e+07,3.329166e+05,-0.006769
2023-12-27,6.530635e+07,-1.942331e-03,6.260414e+09,0.203208,9.390620e+06,0.000305,6.496078e+07,3.455707e+05,0.003480


In [54]:
alphaGen_report["cum_return"] = alphaGen_report["return"].cumsum()
alphaGen_report

,account,return,total_turnover,turnover,total_cost,cost,value,cash,bench,cum_return
datetime,,,,,,,,,,
2022-07-01,1.000000e+08,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,1.000000e+08,-0.004078,0.000000e+00
2022-07-04,9.985780e+07,-4.778849e-16,9.480169e+07,0.948017,1.422025e+05,0.001422,9.480169e+07,5.056106e+06,0.006562,-4.778849e-16
2022-07-05,9.914609e+07,-6.782694e-03,1.177376e+08,0.229686,1.766064e+05,0.000345,9.844103e+07,7.050580e+05,-0.001444,-6.782694e-03
2022-07-06,9.773149e+07,-1.396534e-02,1.377296e+08,0.201641,2.065944e+05,0.000302,9.720178e+07,5.297165e+05,-0.014605,-2.074803e-02
2022-07-07,9.828185e+07,5.930105e-03,1.571936e+08,0.199158,2.357904e+05,0.000299,9.774347e+07,5.383801e+05,0.004408,-1.481793e-02
...,...,...,...,...,...,...,...,...,...,...
2023-12-25,6.592298e+07,9.081655e-03,6.234346e+09,0.198622,9.351519e+06,0.000298,6.558608e+07,3.368954e+05,0.003062,-2.856137e-01
2023-12-26,6.545343e+07,-6.832115e-03,6.247113e+09,0.193660,9.370669e+06,0.000290,6.512052e+07,3.329166e+05,-0.006769,-2.924458e-01
2023-12-27,6.530635e+07,-1.942331e-03,6.260414e+09,0.203208,9.390620e+06,0.000305,6.496078e+07,3.455707e+05,0.003480,-2.943881e-01


In [55]:
import pickle

file_path = f'out/backtests/{ex_num}/boot/0-graph.pkl'

with open(file_path, 'rb') as file:
    chart = pickle.load(file)
chart.show()

In [56]:
import pickle

file_path = f'out/backtests/{ex_num}/boot/0-report.pkl'

with open(file_path, 'rb') as file:
    boot_report = pickle.load(file)
boot_report

,account,return,total_turnover,turnover,total_cost,cost,value,cash,bench
datetime,,,,,,,,,
2022-07-01,1.000000e+08,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,1.000000e+08,-0.004078
2022-07-04,9.985776e+07,-1.204899e-16,9.482534e+07,0.948253,1.422380e+05,0.001422,9.482534e+07,5.032422e+06,0.006562
2022-07-05,9.886206e+07,-9.627935e-03,1.176747e+08,0.228819,1.765121e+05,0.000343,9.815480e+07,7.072671e+05,-0.001444
2022-07-06,9.767159e+07,-1.175821e-02,1.363642e+08,0.189046,2.045463e+05,0.000284,9.716978e+07,5.018065e+05,-0.014605
2022-07-07,9.765739e+07,1.501085e-04,1.556072e+08,0.197018,2.334108e+05,0.000296,9.716261e+07,4.947786e+05,0.004408
...,...,...,...,...,...,...,...,...,...
2023-12-25,7.360360e+07,7.166677e-03,6.514945e+09,0.202917,9.772417e+06,0.000304,7.322060e+07,3.830006e+05,0.003062
2023-12-26,7.322441e+07,-4.845658e-03,6.529967e+09,0.204101,9.794951e+06,0.000306,7.283254e+07,3.918657e+05,-0.006769
2023-12-27,7.338855e+07,2.539878e-03,6.544526e+09,0.198827,9.816789e+06,0.000298,7.300945e+07,3.790992e+05,0.003480


In [57]:
import pickle

file_path = f'out/backtests/{ex_num}/mcts/0-graph.pkl'

with open(file_path, 'rb') as file:
    chart = pickle.load(file)
chart.show()

In [58]:
import pickle

file_path = f'out/backtests/{ex_num}/mcts/0-report.pkl'

with open(file_path, 'rb') as file:
    riskminer_report = pickle.load(file)
riskminer_report

,account,return,total_turnover,turnover,total_cost,cost,value,cash,bench
datetime,,,,,,,,,
2022-07-01,1.000000e+08,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,1.000000e+08,-0.004078
2022-07-04,9.985794e+07,-4.391768e-16,9.470731e+07,0.947073,1.420610e+05,0.001421,9.470731e+07,5.150626e+06,0.006562
2022-07-05,9.955284e+07,-2.702873e-03,1.181686e+08,0.234947,1.772529e+05,0.000352,9.882677e+07,7.260710e+05,-0.001444
2022-07-06,9.840034e+07,-1.128920e-02,1.372590e+08,0.191761,2.058885e+05,0.000288,9.788578e+07,5.145575e+05,-0.014605
2022-07-07,9.937613e+07,1.024179e-02,1.585924e+08,0.216802,2.378886e+05,0.000325,9.879564e+07,5.804947e+05,0.004408
...,...,...,...,...,...,...,...,...,...
2023-12-25,6.808035e+07,5.127972e-03,6.353476e+09,0.208446,9.530214e+06,0.000313,6.771645e+07,3.639016e+05,0.003062
2023-12-26,6.763352e+07,-6.265740e-03,6.366983e+09,0.198393,9.550474e+06,0.000298,6.728150e+07,3.520229e+05,-0.006769
2023-12-27,6.757409e+07,-5.763649e-04,6.380615e+09,0.201554,9.570922e+06,0.000302,6.721969e+07,3.544002e+05,0.003480


In [59]:
import pickle

file_path = f'out/backtests/{ex_num}/emcts/0-report.pkl'

with open(file_path, 'rb') as file:
    eminer_report = pickle.load(file)
eminer_report

,account,return,total_turnover,turnover,total_cost,cost,value,cash,bench
datetime,,,,,,,,,
2022-07-01,1.000000e+08,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,1.000000e+08,-0.004078
2022-07-04,9.985799e+07,-2.182787e-17,9.467659e+07,0.946766,1.420149e+05,0.001420,9.467659e+07,5.181390e+06,0.006562
2022-07-05,9.921861e+07,-6.056581e-03,1.177247e+08,0.230809,1.765870e+05,0.000346,9.851096e+07,7.076560e+05,-0.001444
2022-07-06,9.789941e+07,-1.301136e-02,1.365492e+08,0.189728,2.048238e+05,0.000285,9.738000e+07,5.194045e+05,-0.014605
2022-07-07,9.830644e+07,4.438265e-03,1.548618e+08,0.187055,2.322927e+05,0.000281,9.781685e+07,4.895889e+05,0.004408
...,...,...,...,...,...,...,...,...,...
2023-12-25,7.302298e+07,8.319106e-03,6.608311e+09,0.217604,9.912466e+06,0.000326,7.260985e+07,4.131300e+05,0.003062
2023-12-26,7.230531e+07,-9.532357e-03,6.622705e+09,0.197115,9.934057e+06,0.000296,7.192508e+07,3.802294e+05,-0.006769
2023-12-27,7.222134e+07,-8.761618e-04,6.636449e+09,0.190083,9.954673e+06,0.000285,7.185835e+07,3.629910e+05,0.003480


In [60]:
import pickle

file_path = f'out/backtests/{ex_num}/oracle/0-report.pkl'

with open(file_path, 'rb') as file:
    oracle_report = pickle.load(file)
oracle_report

,account,return,total_turnover,turnover,total_cost,cost,value,cash,bench
datetime,,,,,,,,,
2022-07-01,1.000000e+08,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,1.000000e+08,-0.004078
2022-07-04,9.985784e+07,5.238689e-18,9.477375e+07,0.947738,1.421606e+05,0.001422,9.477375e+07,5.084085e+06,0.006562
2022-07-05,9.964272e+07,-1.817540e-03,1.171878e+08,0.224459,1.757816e+05,0.000337,9.894510e+07,6.976235e+05,-0.001444
2022-07-06,9.803856e+07,-1.580587e-02,1.366715e+08,0.195536,2.050072e+05,0.000293,9.749516e+07,5.433975e+05,-0.014605
2022-07-07,9.893184e+07,9.414484e-03,1.564685e+08,0.201931,2.347028e+05,0.000303,9.841195e+07,5.198983e+05,0.004408
...,...,...,...,...,...,...,...,...,...
2023-12-22,6.650956e+07,2.712098e-03,5.799179e+09,0.195978,8.698768e+06,0.000294,6.616679e+07,3.427763e+05,0.001909
2023-12-25,6.700606e+07,7.755877e-03,5.812075e+09,0.193903,8.718113e+06,0.000291,6.666150e+07,3.445615e+05,0.003062
2023-12-26,6.677282e+07,-3.207625e-03,5.824280e+09,0.182152,8.736421e+06,0.000273,6.645790e+07,3.149191e+05,-0.006769


In [61]:
import pandas as pd
df_com = pd.DataFrame()

df_com["GP"] = gp_report["return"].cumsum()
df_com["Alpha Gen"] = alphaGen_report["return"].cumsum()
df_com["Bootstrapped DQN"] = boot_report["return"].cumsum()
df_com["Oracle"] = oracle_report["return"].cumsum()
df_com["MCTS"] = riskminer_report["return"].cumsum()
df_com["EMCTS"] = eminer_report["return"].cumsum()
df_com["Benchmark"] = boot_report["bench"].cumsum()


df_com.head()

,GP,Alpha Gen,Bootstrapped DQN,Oracle,MCTS,EMCTS,Benchmark
datetime,,,,,,,
2022-07-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-0.004078
2022-07-04,-1.792796e-16,-4.778849e-16,-1.204899e-16,5.238689e-18,-4.391768e-16,-2.182787e-17,0.002484
2022-07-05,-9.364478e-03,-6.782694e-03,-9.627935e-03,-1.817540e-03,-2.702873e-03,-6.056581e-03,0.001040
2022-07-06,-2.897213e-02,-2.074803e-02,-2.138614e-02,-1.762341e-02,-1.399207e-02,-1.906794e-02,-0.013565
2022-07-07,-2.269216e-02,-1.481793e-02,-2.123604e-02,-8.208927e-03,-3.750288e-03,-1.462968e-02,-0.009157


In [62]:
import pickle

rmse_files = []
df_rmse = pd.DataFrame()

for model in ["boot","gp","rl","mcts","emcts"]:
    file_path = f'out/backtests/{ex_num}/{model}/0-rmse.pkl'

    if model == "gp":
        file_path = f'out/backtests/{ex_num}/{model}/2-rmse.pkl'

    with open(file_path, 'rb') as file:
        rmse = pickle.load(file)
    
    df_rmse[model] = rmse["rmse"]

df_rmse.rename(columns={"boot": "Bootstrapped DQN", "gp": "GP", "rl": "Alpha Gen", "mcts":"RiskMiner", "emcts":"EMCTS"}, inplace=True)
df_rmse

,Bootstrapped DQN,GP,Alpha Gen,RiskMiner,EMCTS
date,,,,,
2022-07-01,133.737916,134.257029,132.576987,136.247460,134.071552
2022-07-04,143.107553,158.715492,143.260403,146.499736,144.279692
2022-07-05,151.575413,153.644028,153.530876,151.458931,150.142708
2022-07-06,152.862820,133.121198,149.393332,152.485243,154.867649
2022-07-07,148.507233,163.628401,147.040792,144.626279,147.612334
...,...,...,...,...,...
2023-12-25,144.681630,151.705084,148.075320,132.744135,143.671352
2023-12-26,156.123044,161.195491,156.474969,157.449813,159.359557
2023-12-27,133.575445,124.312622,124.256167,135.365671,122.775580


In [63]:
df_rmse_ma = df_rmse.rolling(30).mean()

In [65]:
df_com.columns

Index(['GP', 'Alpha Gen', 'Bootstrapped DQN', 'Oracle', 'MCTS', 'EMCTS',
       'Benchmark'],
      dtype='object')

In [68]:
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=2, cols=1,subplot_titles=["Cumulative Return", "RMSE"])


for col in ['GP', 'Alpha Gen', 'Bootstrapped DQN', 'MCTS', 'EMCTS', 'Benchmark']:#df_com.columns:
    fig.add_trace(
        go.Scatter(
            x=df_com.index,   
            y=df_com[col],    
            mode='lines',
            name=col,
            legendgroup = '1',
        ),
        row=1, 
        col=1
    )

for col in ['GP', 'Alpha Gen', 'Bootstrapped DQN', 'RiskMiner', 'EMCTS']:#df_rmse_ma.columns:
    fig.add_trace(
        go.Scatter(
            x=df_rmse_ma.index,
            y=df_rmse_ma[col],    
            mode='lines',
            name=col,
            legendgroup = '2',
        ),
        row=2, 
        col=1
    )

# Update the layout to add the title and template
fig.update_layout(
    template='seaborn',
    autosize=False,
    width=1200,
    height=1200,
    legend_tracegroupgap=580,
    legend_groupclick="toggleitem"
)

fig.show()


# fig = px.line(df_com, y=["Bootstrapped DQN","Alpha Gen","GP","Benchmark"], 
#             #   x="lifeExp", 
#               title='Cumulative Return',
#               template="seaborn",
#               )
# fig.show()

# Test

In [72]:
from alphagen_qlib.stock_data import StockData

data = StockData(
        instrument="csi300",
        start_time="2020-01-01",
        end_time="2022-01-01"
    )
data

In [ ]:
from qlib.data import D

instruments = data.stock_ids.tolist()

# Determine the proper start and end times for fetching price data.
# Here we use the same dates as in your StockData instance.
start_time = data._dates[data.max_backtrack_days].strftime("%Y-%m-%d")
end_time = data._dates[-data.max_future_days - 1].strftime("%Y-%m-%d")

# Query Qlib to get the closing price for each instrument.
# The field '$close' is used here (adjust if your field naming is different)
price_df = D.features(
    instruments=instruments,
    fields=["$close"],
    start_time="2020-01-01",
    end_time="2022-01-01"
)

price_df = price_df.reorder_levels(order=[1, 0])
price_df

In [ ]:
def compute_oracle_scores(price_df: pd.DataFrame) -> pd.DataFrame:
    # price_df is expected to be a MultiIndex DataFrame with (date, instrument)
    # Unstack to get dates as rows and instruments as columns
    price_unstacked = price_df.unstack(level=1)
    # Compute daily percentage returns and shift so that prediction on day t 
    # is compared with return from t to t+1
    oracle_signal = price_unstacked.pct_change().shift(-1)
    # Stack back to a MultiIndex DataFrame
    return oracle_signal.stack()


oracle_scores = compute_oracle_scores(price_df)
oracle_scores

C:\Users\tywat\AppData\Local\Temp\ipykernel_29000\2809321555.py:7: FutureWarning:

The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.

C:\Users\tywat\AppData\Local\Temp\ipykernel_29000\2809321555.py:9: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



$close
datetime   instrument          
2020-01-02 SH600000    0.010406
           SH600004   -0.007961
           SH600009   -0.000765
           SH600010    0.008123
           SH600011    0.000000
...                         ...
2021-12-30 SZ300782   -0.024249
           SZ300866   -0.033949
           SZ300888    0.008192
           SZ300896   -0.012722
           SZ300999    0.000476

[197057 rows x 1 columns]

In [17]:
from alphagen_qlib.utils import load_alpha_pool_by_path

calc = QLibStockDataCalculator(data, None)

for p in Path("out/boot_dqn").iterdir():
        inst, size, seed, time, ver = p.name.split('_', 4)
        size, seed = int(size), int(seed)
        if inst != "csi300" or size != 20 or time < "20240923" or ver == "llm_d5":
            continue
        try:
            exprs, weights = load_alpha_pool_by_path(str(p / "249500_steps_pool.json"))
        except:
            continue

boot_score = data.make_dataframe(calc.make_ensemble_alpha(exprs, weights))
boot_score

0
datetime   instrument          
2020-01-02 SH600000    0.029499
           SH600004    0.065036
           SH600009   -0.070193
           SH600010    0.009689
           SH600011    0.194273
...                         ...
2021-12-31 SZ300782   -0.121243
           SZ300866   -0.022676
           SZ300888    0.029904
           SZ300896   -0.026693
           SZ300999    0.017917

[200718 rows x 1 columns]

In [18]:
def normalize_series(series: pd.Series) -> pd.Series:
    return (series - series.mean()) / series.std()

def rank_series_per_date(series: pd.Series) -> pd.Series:
    """
    Rank the series for each date (assumed to be the first level of the MultiIndex).
    The highest value is assigned rank 1.
    """
    return series.groupby(level=0).rank(ascending=False, method='min')
def compute_rmse_per_date(model_scores: pd.Series, oracle_scores: pd.Series) -> pd.DataFrame:
    """
    Compute the RMSE across stocks for each date.
    
    Parameters:
      model_scores: pd.Series with MultiIndex (date, instrument) containing your model's prediction scores.
      oracle_scores: pd.Series with MultiIndex (date, instrument) containing the oracle's prediction scores.
      
    Returns:
      A DataFrame with the date as the index and a column 'rmse' containing the RMSE for that date.
    """
    # normalize
    # model_scores = normalize_series(model_scores)
    # oracle_scores = normalize_series(oracle_scores)

    # rank the scores
    model_scores = rank_series_per_date(model_scores)
    oracle_scores = rank_series_per_date(oracle_scores)

    # Combine both series into one DataFrame
    df = pd.DataFrame({
        "model": model_scores,
        "oracle": oracle_scores
    })
    # Group by the date level. If your MultiIndex doesn't have names,
    # you can group by level=0 (assuming the first level is the date).
    rmse_series = df.groupby(level=0).apply(
        lambda group: np.sqrt(((group["oracle"] - group["model"]) ** 2).mean())
    )
    rmse_df = rmse_series.to_frame(name="rmse")
    # Ensure the index is named "date" (or adjust as needed)
    rmse_df.index.name = "date"
    return rmse_df

rmse_df = compute_rmse_per_date(boot_score.iloc[:,0], oracle_scores.iloc[:,0])
rmse_df

,rmse
date,
2020-01-02,175.373923
2020-01-03,172.032975
2020-01-06,158.534962
2020-01-07,143.087359
2020-01-08,158.674098
...,...
2021-12-27,165.069690
2021-12-28,162.478913
2021-12-29,165.498930


In [19]:
import plotly.express as px
fig = px.line(rmse_df, y=["rmse"], 
            #   x="lifeExp", 
              title='RMSE',
              template="seaborn",
              )
fig.show()